In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Data

# Credit Card Lead Prediction
Happy Customer Bank is a mid-sized private bank that deals in all kinds of banking products, like Savings accounts, Current accounts, investment products, credit products, among other offerings.The bank also cross-sells products to its existing customers and to do so they use different kinds of communication like tele-calling, e-mails, recommendations on net banking, mobile banking, etc. In this case, the Happy Customer Bank wants to cross sell its credit cards to its existing customers. The bank has identified a set of customers that are eligible for taking these credit cards. Now, the bank is looking for your help in identifying customers that could show higher intent towards a recommended credit card, given:
Customer details (gender, age, region etc.)
Details of his/her relationship with the bank (Channel_Code,Vintage, 'Avg_Asset_Value etc.)
Data Dictionary
Train Data
Variable           Definition

ID                Unique Identifier for a row

Gender            Gender of the Customer

Age                Age of the Customer (in Years)

Region_Code        Code of the Region for the customers

Occupatio           Occupation Type for the customer

Channel_Code        Acquisition Channel Code for the Customer  (Encoded)

Vintag              Vintage for the Customer (In Months)

Credit_Product       If the Customer has any active credit product (Home loan,
                       Personal loan, Credit Card etc.)

Avg_Account_Balance   Average Account Balance for the Customer in last 12 Months

Is_Active             If the Customer is Active in last 3 Months

Is_Lead(Target)       Customer is interested for the Credit Card 0:Customer is not interested 1:Customer is   interested


Test Dat              

Variable                   Definition

ID                   Unique Identifier for a row

Gender
                     Gender of the Customer

Age
                     Age of the Customer (in Years)

Region_Code          Code of the Region for the customers

Occupatio            Occupation Type for the customer

Channel_Code         Acquisition Channel Code for the Customer  (Encoded)

Vintage              Vintage for the Customer (In Months)

Credit_Product       If the Customer has any active credit product (Home loan,Personal loan, Credit Card etc.)

Avg_Account_Balance  Average Account Balance for the Customer in last 12 Months

Is_Active            If the Customer is Active in last 3 Months

# List Of Hypothesis:
1) Do age plays a significant role in deciding whether one will take creait card service or not?
2) Do duration of association of customers matter in activating credit card service?
3) Can avg account balance play a role in deciding whether customer will take credit card or not?
4) Do credit card issude or not depends on gender?
5) Is credit of any type leads to take credit card?
6) Do occupation of individuals plays an important role in deciding whether one will take credit card or not?
7) Do active customers have high frequency of taking credit card service?
8) Are acqistion channel an important source in deciding whether customer will take service or not?
9) Is region of customer an important factor in deciding the activation of service?

In [ ]:
#Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pylab as py
import warnings
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import KFold, StratifiedKFold,GroupKFold
from lightgbm import LGBMClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsRegressor,KNeighborsClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import mean_squared_error,r2_score,roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import gc
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, roc_curve, precision_score, recall_score, precision_recall_curve
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.filterwarnings('ignore')
sns.set(style = 'white')

In [ ]:
train = pd.read_csv("../input/jobathon-may-2021-credit-card-lead-prediction/train.csv")
test = pd.read_csv("../input/jobathon-may-2021-credit-card-lead-prediction/test.csv")

In [ ]:
train.columns, test.columns

In [ ]:
train.head(10)

In [ ]:
train.dtypes

#Here we can see that the variables like 'ID', 'Gender', 'Region_Code', 'Occupation', 'Channel_Code' are object type datatype. This means python in not able to recognise the data type of these variables. So, we will have to find their right data type and assign them a new data type.

In [ ]:
# As most of the features are categorical, check if test data has all categorical values same as train data.
# If any column has new values then we should be careful using that feature in training directly..

count=0
for col in train.columns:
    if col not in ['ID','Age','Vintage','Avg_Account_Balance','Is_Lead']:
        for val in test[col].unique():
            if val not in train[col].unique():
                print(col,val)
                count+=1

In [ ]:
test_region_list=test['Region_Code'].tolist()

In [ ]:
train=train[train['Region_Code'].isin(test_region_list)]

In [ ]:
train.isnull().sum(), test.isnull().sum()

In [ ]:
# Lets check if data is uniformly distributed between train and test. Also,if there are any outliers or not.
train.describe()

In [ ]:
test.describe()

In [ ]:
def UVA_outlier(data, var_group, include_outlier = True):
  '''
  Univariate_Analysis_outlier:
  takes a group of variables (INTEGER and FLOAT) and plot/print boplot and descriptives\n
  Runs a loop: calculate all the descriptives of i(th) variable and plot/print it \n\n

  data : dataframe from which to plot from\n
  var_group : {list} type Group of Continuous variables\n
  include_outlier : {bool} whether to include outliers or not, default = True\n
  '''

  size = len(var_group)
  plt.figure(figsize = (7*size,4), dpi = 100)
  
  #looping for each variable
  for j,i in enumerate(var_group):
    
    # calculating descriptives of variable
    quant25 = data[i].quantile(0.25)
    quant75 = data[i].quantile(0.75)
    IQR = quant75 - quant25
    med = data[i].median()
    whis_low = med-(1.5*IQR)
    whis_high = med+(1.5*IQR)

    # Calculating Number of Outliers
    outlier_high = len(data[i][data[i]>whis_high])
    outlier_low = len(data[i][data[i]<whis_low])

    if include_outlier == True:
      print(include_outlier)
      #Plotting the variable with every information
      plt.subplot(1,size,j+1)
      sns.boxplot(data[i], orient="v")
      plt.ylabel('{}'.format(i))
      plt.title('With Outliers\nIQR = {}; Median = {} \n 2nd,3rd  quartile = {};\n Outlier (low/high) = {} \n'.format(
                                                                                                   round(IQR,2),
                                                                                                   round(med,2),
                                                                                                   (round(quant25,2),round(quant75,2)),
                                                                                                   (outlier_low,outlier_high)
                                                                                                   ))
      
    else:
      # replacing outliers with max/min whisker
      train = data[var_group][:]
      train[i][train[i]>whis_high] = whis_high+1
      train[i][train[i]<whis_low] = whis_low-1
      
      # plotting without outliers
      plt.subplot(1,size,j+1)
      sns.boxplot(train[i], orient="v")
      plt.ylabel('{}'.format(i))
      plt.title('Without Outliers\nIQR = {}; Median = {} \n 2nd,3rd  quartile = {};\n Outlier (low/high) = {} \n'.format(
                                                                                                   round(IQR,2),
                                                                                                   round(med,2)
                                                                                                   (round(quant25,2),round(quant75,2)),
                                                                                                   (outlier_low,outlier_high)
                                                                                                   ))  



In [ ]:
num_cols = ['Age', 'Vintage','Avg_Account_Balance']

In [ ]:
UVA_outlier(train, num_cols)

In [ ]:
# There seems to be large no of outliers in variable "Avg_Account_Balance". Lets also check for outliers in test["Avr_Account_Balance"]
test.boxplot(column = ['Avg_Account_Balance'])

# we can see that both test train variable avg_account_balance has outliers on the right hand side.

In [ ]:
#Bivariate Analysis
#Continuius Continuous
numericals = train[['Age', 'Vintage','Avg_Account_Balance']]
correlation = numericals.corr()
correlation

In [ ]:
plt.figure(figsize=(36,6), dpi=140)
for j,i in enumerate(['pearson','kendall','spearman']):
  plt.subplot(1,3,j+1)
  correlation = numericals.dropna().corr(method=i)
  sns.heatmap(correlation, linewidth = 2)
  plt.title(i, fontsize=18)

# There seems to be no high correlation betweem continuous variables.

In [ ]:
#BIVARIATE ANALYSIS: CONTINUOUS CATEGORICAL VARIABLES

#List of Hypothesis and investigation to perform under this combination.  
#1) Do age plays a significant role in deciding whether one will take creait card service or not?
#2) Do duration of association of customers matter in activating credit card service?
#3) Can avg account balance play a role in deciding whether customer will take credit card or not?
  
def TwoSampZ(X1, X2, sigma1, sigma2, N1, N2):
  '''
  takes mean, standard deviation, and number of observations and returns p-value calculated for 2-sampled Z-Test
  '''
  from numpy import sqrt, abs, round
  from scipy.stats import norm
  ovr_sigma = sqrt(sigma1**2/N1 + sigma2**2/N2)
  z = (X1 - X2)/ovr_sigma
  pval = 2*(1 - norm.cdf(abs(z)))
  return pval
def TwoSampT(X1, X2, sd1, sd2, n1, n2):
  '''
  takes mean, standard deviation, and number of observations and returns p-value calculated for 2-sample T-Test
  '''
  from numpy import sqrt, abs, round
  from scipy.stats import t as t_dist
  ovr_sd = sqrt(sd1**2/n1 + sd2**2/n2)
  t = (X1 - X2)/ovr_sd
  df = n1+n2-2
  pval = 2*(1 - t_dist.cdf(abs(t),df))
  return pval

def Bivariate_cont_cat(train, cont, cat, category):
  #creating 2 samples
  x1 = train[cont][train[cat]==category][:]
  x2 = train[cont][~(train[cat]==category)][:]
  
  #calculating descriptives
  n1, n2 = x1.shape[0], x2.shape[0]
  m1, m2 = x1.mean(), x2.mean()
  std1, std2 = x1.std(), x2.mean()
  
  #calculating p-values
  t_p_val = TwoSampT(m1, m2, std1, std2, n1, n2)
  z_p_val = TwoSampZ(m1, m2, std1, std2, n1, n2)

  #table
  table = pd.pivot_table(data=train, values=cont, columns=cat, aggfunc = np.mean)

  #plotting
  plt.figure(figsize = (15,6), dpi=140)
  
  #barplot
  plt.subplot(1,2,1)
  sns.barplot([str(category),'not {}'.format(category)], [m1, m2])
  plt.ylabel('mean {}'.format(cont))
  plt.xlabel(cat)
  plt.title('t-test p-value = {} \n z-test p-value = {}\n {}'.format(t_p_val,
                                                                z_p_val,
                                                                table))

  # boxplot
  plt.subplot(1,2,2)
  sns.boxplot(x=cat, y=cont, data=train)
  plt.title('categorical boxplot')

In [ ]:
Bivariate_cont_cat(train, 'Age','Is_Lead', 1)

There seems to be significant difference between the age of person who are interested or not interested for credit card.


In [ ]:
Bivariate_cont_cat(train, 'Vintage','Is_Lead', 1)

Duration of association plays an important role in deciding whether a customer will take or not take credit card.

In [ ]:
Bivariate_cont_cat(train, 'Avg_Account_Balance','Is_Lead', 1)

Avg_Account_Balacne of customers also plays an important role in deciding whether customer will avail the service of credit card or not.

In [ ]:
##List of all the hypothesis:-
#1 4) Do credit card issude or not depends on gender?
#2) Is credit of any type leads to take credit card?
#3) Do occupation of individuals plays an important role in deciding whether one will take credit card or not?
#4) Do active customers have high frequency of taking credit card service?
#5) Are acqistion channel an important source in deciding whether customer will take service or not?
#6) Is region of customer an important factor in deciding the activation of service?
#Bivariate : Categorical-Categorical
def BVA_categorical_plot(data, tar, cat):
  '''
  take data and two categorical variables,
  calculates the chi2 significance between the two variables 
  and prints the result with countplot & CrossTab
  '''
  #isolating the variables
  data = train[[cat,tar]][:]

  #forming a crosstab
  table = pd.crosstab(train[tar],train[cat],)
  f_obs = np.array([table.iloc[0][:].values,
                    table.iloc[1][:].values])

  #performing chi2 test
  from scipy.stats import chi2_contingency
  chi, p, dof, expected = chi2_contingency(f_obs)
  
  #checking whether results are significant
  if p<0.05:
    sig = True
  else:
    sig = False

  #plotting grouped plot
  sns.countplot(x=cat, hue=tar, data= train)
  plt.title("p-value = {}\n difference significant? = {}\n".format(round(p,8),sig))

  #plotting percent stacked bar plot
  #sns.catplot(ax, kind='stacked')
  ax1 = train.groupby(cat)[tar].value_counts(normalize=True).unstack()
  ax1.plot(kind='bar', stacked='True',title=str(ax1))
  int_level = train[cat].value_counts()
  

In [ ]:
BVA_categorical_plot(train, 'Gender', 'Is_Lead')

In [ ]:
BVA_categorical_plot(train, 'Region_Code', 'Is_Lead')

In [ ]:
BVA_categorical_plot(train, 'Credit_Product', 'Is_Lead')

In [ ]:
BVA_categorical_plot(train, 'Is_Active', 'Is_Lead')

In [ ]:
BVA_categorical_plot(train, 'Channel_Code', 'Is_Lead')

In [ ]:
BVA_categorical_plot(train, 'Occupation', 'Is_Lead')

We can see for all the categorical variables  each categorical category are statistically significant i.e. each category decides whether credit card will be issued or not

In [ ]:
# join both train and test data for preprocessing..
train['train'] = 1
test['train'] = 0

df = pd.concat([train,test],axis=0).reset_index(drop=True)

In [ ]:
# check about the data type of each feature
df.info()

In [ ]:
#check for missing values
df.isnull().sum()

In [ ]:
df['Credit_Product'].value_counts(normalize = True)

In [ ]:
# we have assigned a new category to the missing values of the credit_product because it might be that there association with bank might be of different nature?
df['Credit_Product'] = df['Credit_Product'].fillna('X')

In [ ]:
le = LabelEncoder()
for col in ['Region_Code']:
    df[col]=  df[col].astype('str')
    df[col]= le.fit_transform(df[col])
       

In [ ]:
# Introduced dummy fot the nominal categorical varibales
df= pd.concat([df,pd.get_dummies(df['Gender'],prefix = str('Gender'),prefix_sep='_')],axis = 1)
df = pd.concat([df,pd.get_dummies(df['Occupation'],prefix = str('Occupation'),prefix_sep='_')],axis = 1)
df = pd.concat([df,pd.get_dummies(df['Channel_Code'],prefix = str('Channel_Code'),prefix_sep='_')],axis = 1)
df = pd.concat([df,pd.get_dummies(df['Credit_Product'],prefix = str('Credit_Product'),prefix_sep='_')],axis = 1)
df = pd.concat([df,pd.get_dummies(df['Is_Active'],prefix = str('Is_Active'),prefix_sep='_')],axis = 1)

In [ ]:
df.drop(['Gender'], axis = 1, inplace = True)
df.drop(['Occupation'], axis = 1, inplace = True)
df.drop(['Channel_Code'], axis = 1, inplace = True)
df.drop(['Credit_Product'], axis =1 , inplace = True)
df.drop(['Is_Active'], axis =1 , inplace = True)

In [ ]:
df.head()

In [ ]:
cat_features = ['Gender_Female','Gender_Male','Occupation_Entrepreneur','Occupation_Other','Occupation_Salaried','Occupation_Self_Employed','Channel_Code_X1','Channel_Code_X2','Channel_Code_X3','Channel_Code_X4','Credit_Product_No','Credit_Product_X','Credit_Product_Yes','Is_Active_No','Is_Active_Yes']

In [ ]:
seed = 0

In [ ]:
df.drop(['ID'], axis = 1, inplace = True)

In [ ]:
df.head()

In [ ]:
cat_features = ['Gender_Female','Gender_Male','Occupation_Entrepreneur','Occupation_Other','Occupation_Salaried','Occupation_Self_Employed','Channel_Code_X1','Channel_Code_X2','Channel_Code_X3','Channel_Code_X4','Credit_Product_No','Credit_Product_X','Credit_Product_Yes','Is_Active_No','Is_Active_Yes']

In [ ]:
#I have used LightGBM model for training the data. I have used stratified Kfold(10) as cross validation technique.


def train_lgbm(df,seed,cat_features):
    X = df[df['train']==1]
    y = df[df['train']==1]['Is_Lead']
    test_data = df[df['train']==0]
    num_split = 10
    folds = StratifiedKFold(n_splits=num_split)
    excluded_features = ['Is_Lead','train','ID']
    train_features = [_f for _f in X.columns if _f not in excluded_features]
    importances = pd.DataFrame()
    oof_reg_preds = np.zeros(X.shape[0])
    test_preds = np.zeros(test_data.shape[0])
    for fold_, (trn_, val_) in enumerate(folds.split(X, y)):
        print("Fold:",fold_)
        trn_x, trn_y = X[train_features].iloc[trn_], y.iloc[trn_]
        val_x, val_y = X[train_features].iloc[val_], y.iloc[val_]
        clf1 = LGBMClassifier(
            n_jobs=-1,
            learning_rate=0.0094,
            n_estimators=10000,
            colsample_bytree=0.94,
            subsample = 0.75,
            subsample_freq = 1,
            reg_alpha= 1.0,
            reg_lambda = 5.0,
            random_state=seed
        )
        clf1.fit(
            trn_x,trn_y ,
            eval_set=[(val_x, val_y)],
            early_stopping_rounds=100,
            verbose=False,
            eval_metric='auc',
            categorical_feature = cat_features
        )
        
        # Extra boosting.
        clf = LGBMClassifier(
            n_jobs=-1,
            learning_rate=0.00094,
            n_estimators=10000,
            colsample_bytree=0.94,
            subsample = 0.75,
            subsample_freq = 1,
            reg_alpha= 1.0,
            reg_lambda = 5.0,
            random_state=seed
        )
        clf.fit(
            trn_x,trn_y ,
            eval_set=[(val_x, val_y)],
            early_stopping_rounds=300,
            verbose=False,
            eval_metric='auc',
            categorical_feature = cat_features,
            init_model = clf1
        )
        
        imp_df = pd.DataFrame()
        imp_df['feature'] = train_features
        imp_df['importance'] = clf.booster_.feature_importance(importance_type='gain')

        imp_df['fold'] = fold_ + 1
        importances = pd.concat([importances, imp_df], axis=0, sort=False)

        oof_reg_preds[val_] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)[:, 1]
        _preds = clf.predict_proba(test_data[train_features], num_iteration=clf.best_iteration_)[:, 1]
        test_preds += _preds
        print("fold"+str(fold_)+" auc",roc_auc_score(val_y, oof_reg_preds[val_]))
        del  trn_x, trn_y, val_x, val_y,trn_, val_
        gc.collect()

    test_preds = test_preds/num_split
    print(roc_auc_score(y, oof_reg_preds))
    
    return test_preds,importances,oof_reg_preds,clf,train_features

In [ ]:
# train the model by passing data, random seed as 42 and categorical feature list
lgbm_preds,feat_imp_lgbm,oof_lgbm,lgbm_model,feats = train_lgbm(df,42,cat_features)

In [ ]:
lgbm_preds1,feat_imp_lgbm1,oof_lgbm1,lgbm_model1,feats = train_lgbm(df,11,cat_features)

In [ ]:
# Function to display feature importance...
def display_importances(feature_importance_df_,model):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:30].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(12, 8))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title(model+" Features (avg over folds)")
    plt.tight_layout()
    plt.savefig(model +"_importances-01.png")

In [ ]:
display_importances(feat_imp_lgbm,"LGBM")

In [ ]:
oof_ensemble = oof_lgbm*.50 + oof_lgbm1*.50
roc_auc_score(df[df['train']==1]['Is_Lead'],oof_ensemble)

In [ ]:
final_preds = lgbm_preds*.50 + lgbm_preds1*.50